In [12]:
import glob
import pandas as pd
import numpy as np
import os
from datetime import date, datetime, time, timedelta
from collections import defaultdict

## Preparacion de datos:

Se utilizan como features:

* datos medidos a las 19 hrs (aproximadamente hora de la puesta de sol, esto es perfectible).
* temperatura medida a las 14 hrs del mismo dia.

y se predice:

* si ocurre o no una helada durante la noche de dicho dia, es decir, en el periodo de 19 hrs hasta las 19 hrs del dia siguiente.

Los cuales son extraidos arbitrariamente de diversas estaciones, donde los datos se agrupan por dia.

In [13]:
path = "../data/cleared"
dfs = []
dict_meta = {"region": [], "estacion": [], "codigo": []}
for file in glob.glob(path + "/*/*"):
    print(file, end="\r")
    file = file.replace("\\", "/")
    dict_meta["region"].append(file.split("/")[-2])
    dict_meta["estacion"].append(file.split("/")[-1].split(".")[0])
    df = pd.read_csv(file)
    dict_meta["codigo"].append(df["Codigo"][0])
    dfs.append(df)
    

In [14]:
df = pd.concat(dfs, ignore_index=True)
df = df.sort_values(by=['Codigo', 'Fecha', 'Hora'])
df = df.reset_index(drop=True)

In [16]:
len(np.unique(df["Codigo"]))

162

In [4]:
temp = np.array(df["Temperatura"])
code = np.array(df["Codigo"])
ddate = np.array(df["Fecha"])
dtime = np.array(df["Hora"])

ddtt = []
displaced_date = []
# create datetime object and displaceit by -19 hours
deltatime = timedelta(hours = 19)
c = 0
print(len(ddate))
for d, t in zip(ddate, dtime):
    
    if c % 10000 == 0:
        print(c, end="\r")
    if " " in d:
        d = d.split(" ")[0]
        datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
    elif "-" in d:
        datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
    else:
        datm = datetime.strptime(d + " " + t, '%d/%m/%Y %H:%M:%S')
    displaced_date.append((datm - deltatime).date().__str__())
    ddtt.append(datm)
    c += 1
    

5301924


In [142]:
df["datetime"] = ddtt
df["displaced_date"] = displaced_date
df.count()

Codigo                 5257742
Fecha                  5257742
Hora                   5257742
Temperatura            5257742
Humedad                5257742
Direccion de Viento    5257742
Velocidad de Viento    5257742
Precipitacion          5257742
Radiacion Solar        5257742
Presion Atmosferica    5257742
datetime               5257742
displaced_date         5257742
dtype: int64

In [143]:
df2v = df.drop_duplicates(subset=['Codigo', 'Fecha', 'Hora'], keep="first")
df2v.count()

Codigo                 5257742
Fecha                  5257742
Hora                   5257742
Temperatura            5257742
Humedad                5257742
Direccion de Viento    5257742
Velocidad de Viento    5257742
Precipitacion          5257742
Radiacion Solar        5257742
Presion Atmosferica    5257742
datetime               5257742
displaced_date         5257742
dtype: int64

In [144]:
df = df2v

In [145]:
# save this new dataframe
outdir = "../data/processed"
if not os.path.exists(outdir):
    os.makedirs(outdir)
df.to_csv(outdir + "/backup_dffull.csv", index=False, header=True)

In [8]:
print("Total de dias")
len(df.drop_duplicates(subset=['Codigo', 'displaced_date'], keep="first"))

Total de dias


56838

In [9]:
df_frost = df[df["Temperatura"] < 0]
df_frost.head()

,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica,datetime,displaced_date
53,13,2010-06-12,01:30:00,-0.1,97.1,251.2,0.0,0.0,0.0,1009.0,2010-06-12 01:30:00,2010-06-11
455,13,2010-06-16,06:00:00,-0.1,84.4,138.9,0.0,0.0,0.0,1028.0,2010-06-16 06:00:00,2010-06-15
456,13,2010-06-16,06:15:00,-0.1,83.3,93.5,0.0,0.0,0.0,1027.0,2010-06-16 06:15:00,2010-06-15
457,13,2010-06-16,06:30:00,-0.2,77.6,126.5,0.0,0.0,0.0,1028.0,2010-06-16 06:30:00,2010-06-15
458,13,2010-06-16,06:45:00,-0.4,76.7,162.7,0.0,0.0,0.0,1028.0,2010-06-16 06:45:00,2010-06-15


In [10]:
df_frost_unique = df_frost.drop_duplicates(subset=['Codigo', 'displaced_date'], keep="first")
print("total de dias con heladas", len(df_frost_unique))
df_frost_unique.head()

total de dias con heladas 3131


,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica,datetime,displaced_date
53,13,2010-06-12,01:30:00,-0.1,97.1,251.2,0.0,0.0,0.0,1009.0,2010-06-12 01:30:00,2010-06-11
455,13,2010-06-16,06:00:00,-0.1,84.4,138.9,0.0,0.0,0.0,1028.0,2010-06-16 06:00:00,2010-06-15
509,13,2010-06-16,19:30:00,-0.4,81.4,201.9,0.0,0.0,0.0,1025.0,2010-06-16 19:30:00,2010-06-16
599,13,2010-06-17,19:00:00,-2.3,84.8,98.9,0.0,0.0,0.0,1031.0,2010-06-17 19:00:00,2010-06-17
695,13,2010-06-18,19:00:00,-1.8,92.8,206.0,0.0,0.0,0.0,1022.0,2010-06-18 19:00:00,2010-06-18


dataframe de temperaturas minimas

In [99]:
df

,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica,datetime,displaced_date
0,13,2010-06-11,12:15:00,6.2,70.2,179.9,0.0,0.0,125.5,1008.0,2010-06-11 12:15:00,2010-06-10
1,13,2010-06-11,12:30:00,6.4,64.8,138.4,0.0,0.0,181.7,1007.0,2010-06-11 12:30:00,2010-06-10
2,13,2010-06-11,12:45:00,6.4,64.9,96.9,0.0,0.0,159.5,1007.0,2010-06-11 12:45:00,2010-06-10
3,13,2010-06-11,13:00:00,6.9,62.1,121.0,0.0,0.0,328.2,1008.0,2010-06-11 13:00:00,2010-06-10
4,13,2010-06-11,13:15:00,7.1,62.0,47.0,0.0,0.0,195.2,1007.0,2010-06-11 13:15:00,2010-06-10
5,13,2010-06-11,13:30:00,7.5,64.2,37.1,0.0,0.0,210.6,1006.0,2010-06-11 13:30:00,2010-06-10
6,13,2010-06-11,13:45:00,7.5,63.3,77.0,0.0,0.0,196.7,1006.0,2010-06-11 13:45:00,2010-06-10
7,13,2010-06-11,14:00:00,7.2,66.7,61.4,0.0,0.0,146.0,1006.0,2010-06-11 14:00:00,2010-06-10
8,13,2010-06-11,14:15:00,6.8,69.5,61.8,0.0,0.0,75.8,1006.0,2010-06-11 14:15:00,2010-06-10
9,13,2010-06-11,14:30:00,7.0,68.0,106.3,0.0,0.0,131.4,1006.0,2010-06-11 14:30:00,2010-06-10


In [105]:
# buscamos los indices donde cambia el dia
df = df.reset_index(drop=True)
idxs = df.drop_duplicates(subset=['Codigo', 'displaced_date'], keep="first").index
matrix = df.values

56838


In [121]:
mints = []
codes = []
disp_dates = []
print(len(idxs))
# obtenemos las temperaturas minimas
for i in range(len(idxs)-1):
    if i % 100 == 0:
        print(i, end="\r")
    ini = idxs[i]
    end = idxs[i+1]
    min_t = np.min(matrix[ini:end,3])
    code = matrix[ini][0]
    displaced_date = matrix[ini][-1]
    mints.append(min_t)
    codes.append(code)
    disp_dates.append(displaced_date)

56838


In [138]:
df_mint = pd.DataFrame({"code": codes, "date": disp_dates, "minT": mints})
df_mint.count()

code    56837
date    56837
minT    56837
dtype: int64

descartamos todo dato que no sea medicion a las 14 o 19 horas

In [11]:
df_14 = df[df["Hora"] == "14:00:00"]
df_19 = df[df["Hora"] == "19:00:00"]
df_cut = pd.concat([df_14, df_19], ignore_index=True)
df_cut = df_cut.sort_values(by=['Codigo', 'Fecha', 'Hora'])
df_cut = df_cut.reset_index(drop=True)
df_cut

,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica,datetime,displaced_date
0,13,2010-06-11,14:00:00,7.2,66.7,61.4,0.0,0.0,146.0,1006.0,2010-06-11 14:00:00,2010-06-10
1,13,2010-06-11,19:00:00,3.8,84.7,146.5,0.0,0.0,0.0,1005.0,2010-06-11 19:00:00,2010-06-11
2,13,2010-06-12,14:00:00,14.0,48.6,315.7,0.0,0.0,246.3,1008.0,2010-06-12 14:00:00,2010-06-11
3,13,2010-06-12,19:00:00,9.7,66.0,266.1,0.0,0.0,0.0,1013.0,2010-06-12 19:00:00,2010-06-12
4,13,2010-06-13,14:00:00,8.6,82.4,228.0,0.0,0.0,118.5,1011.0,2010-06-13 14:00:00,2010-06-12
5,13,2010-06-13,19:00:00,6.3,95.5,141.3,0.0,0.0,0.0,1011.0,2010-06-13 19:00:00,2010-06-13
6,13,2010-06-14,14:00:00,6.0,84.9,53.2,0.0,0.2,53.6,1014.0,2010-06-14 14:00:00,2010-06-13
7,13,2010-06-14,19:00:00,3.7,94.2,114.6,0.0,0.4,0.0,1019.0,2010-06-14 19:00:00,2010-06-14
8,13,2010-06-15,14:00:00,0.8,89.7,240.3,0.0,0.0,39.6,1026.0,2010-06-15 14:00:00,2010-06-14
9,13,2010-06-15,19:00:00,0.7,89.3,144.7,0.0,0.0,0.0,1027.0,2010-06-15 19:00:00,2010-06-15


unimos las filas

In [125]:
data = df_cut.values
i = 0
L = len(data) - 1
gdata = []
while i < L:
    d1 = data[i] 
    d2 = data[i+1]
    if d1[1] == d1[1]:
        par = list(d1[:2])
        par.extend(d1[3:-2])
        par.extend(d2[3:-2])
        gdata.append(par)
        
    i += 2
    
# agregamos target booleano
i = 0
L = len(gdata)
print(L)
while i < L:
    if i % 100 == 0:
        print(i, end="\r")
    ddtt = gdata[i][1]
    code = gdata[i][0]
    df_frost_day = df_frost_unique[(df_frost_unique["Codigo"] == code) & 
                                           (df_frost_unique["displaced_date"] == ddtt)]
    frost = len(df_frost_day) == 1
    gdata[i].append(frost)
    i += 1

55850


In [141]:
gtdata = gdata
# agregamos temperatura minima\
print("AGREGAMOS TEMPERATURA MINIMA")
i = 0
while i < L:
    if i % 100 == 0:
        print(i, end="\r")
    ddtt = gtdata[i][1]
    code = gtdata[i][0]
    min_t = df_mint[(df_mint["code"] == code) & (df_mint["date"] == ddtt)]
    if min_t.empty:
        gtdata = np.delete(gtdata, i, 0)
    else:
        gtdata[i].append(min_t.iloc[0]["minT"])
        i += 1


AGREGAMOS TEMPERATURA MINIMA


KeyboardInterrupt: 

In [137]:
df_mint[(df_mint["code"] == 49)].iloc[50:]

,code,date,minT
820,49,2010-06-08,4.0
821,49,2010-06-09,3.6
822,49,2010-06-10,7.8
823,49,2010-06-11,10.8
824,49,2010-06-12,4.8
825,49,2010-06-13,6.4
826,49,2010-06-14,8.2
827,49,2010-06-15,7.7
828,49,2010-06-16,9.1
829,49,2010-06-17,8.0


In [ ]:
ggdata = np.array(gdata)
print(len(data), len(gdata), len(gdata)*2)

In [90]:
# targets 
targets = [x == "True" for x in ggdata[:,16]]
df_dict = {"code": list(ggdata[:,0].astype(int)), 
           "Fecha": list(ggdata[:,1].astype(str)), 
           "T_14": list(ggdata[:,2].astype(float)), 
           "Hum_14": list(ggdata[:,3].astype(float)), 
           "DVin_14": list(ggdata[:,4].astype(float)), 
           "VVin_14": list(ggdata[:,5].astype(float)), 
           "Prec_14": list(ggdata[:,6].astype(float)), 
           "RadSol_14": list(ggdata[:,7].astype(float)), 
           "PATM_14h": list(ggdata[:,8].astype(float)), 
           "T_19": list(ggdata[:,9].astype(float)), 
           "Hum_19": list(ggdata[:,10].astype(float)), 
           "DVin_19": list(ggdata[:,11].astype(float)), 
           "VVin_19": list(ggdata[:,12].astype(float)), 
           "Prec_19": list(ggdata[:,13].astype(float)), 
           "RadSol_19": list(ggdata[:,14].astype(float)), 
           "PATM_19h": list(ggdata[:,15].astype(float)),
           "Min_Temp": list(gg)
           "Target": targets}

out_df = pd.DataFrame(df_dict)
out_df.head()

,code,Fecha,T_14,Hum_14,DVin_14,VVin_14,Prec_14,RadSol_14,PATM_14h,T_19,Hum_19,DVin_19,VVin_19,Prec_19,RadSol_19,PATM_19h,Target
0,13,2010-06-11,7.2,66.7,61.4,0.0,0.0,146.0,1006.0,3.8,84.7,146.5,0.0,0.0,0.0,1005.0,True
1,13,2010-06-12,14.0,48.6,315.7,0.0,0.0,246.3,1008.0,9.7,66.0,266.1,0.0,0.0,0.0,1013.0,False
2,13,2010-06-13,8.6,82.4,228.0,0.0,0.0,118.5,1011.0,6.3,95.5,141.3,0.0,0.0,0.0,1011.0,False
3,13,2010-06-14,6.0,84.9,53.2,0.0,0.2,53.6,1014.0,3.7,94.2,114.6,0.0,0.4,0.0,1019.0,False
4,13,2010-06-15,0.8,89.7,240.3,0.0,0.0,39.6,1026.0,0.7,89.3,144.7,0.0,0.0,0.0,1027.0,True


In [92]:
# guardamos este dataset procesado
outdir = "../data/processed"
if not os.path.exists(outdir):
    os.makedirs(outdir)
out_df.to_csv(outdir + "/dataset1.csv", index=False, header=True)

In [91]:
# resumen final
out_df[out_df["Target"]]

,code,Fecha,T_14,Hum_14,DVin_14,VVin_14,Prec_14,RadSol_14,PATM_14h,T_19,Hum_19,DVin_19,VVin_19,Prec_19,RadSol_19,PATM_19h,Target
0,13,2010-06-11,7.2,66.7,61.4,0.0,0.0,146.0,1006.0,3.8,84.7,146.5,0.0,0.0,0.0,1005.0,True
4,13,2010-06-15,0.8,89.7,240.3,0.0,0.0,39.6,1026.0,0.7,89.3,144.7,0.0,0.0,0.0,1027.0,True
5,13,2010-06-16,2.3,70.2,183.1,0.0,0.0,278.3,1023.0,0.2,69.1,271.2,0.0,0.0,0.0,1025.0,True
6,13,2010-06-17,4.1,57.9,259.1,0.0,0.0,326.6,1027.0,-2.3,84.8,98.9,0.0,0.0,0.0,1031.0,True
7,13,2010-06-18,4.7,59.6,299.3,0.0,0.0,203.4,1020.0,-1.8,92.8,206.0,0.0,0.0,0.0,1022.0,True
8,13,2010-06-19,7.5,50.2,260.8,0.0,0.0,46.3,1010.0,4.2,49.4,85.5,0.0,0.0,0.0,1017.0,True
9,13,2010-06-20,2.0,68.4,301.1,0.0,0.0,150.6,1020.0,-1.0,79.8,139.1,0.0,0.0,0.0,1020.0,True
10,13,2010-06-21,2.7,80.5,145.4,0.0,0.0,77.5,1012.0,0.1,91.9,113.1,0.0,0.0,0.0,1013.0,True
19,13,2010-06-30,8.3,55.5,320.8,0.0,0.0,128.5,1006.0,5.3,69.0,128.5,0.0,0.0,0.0,1003.0,True
120,24,2010-05-26,6.9,87.9,172.1,0.0,0.0,0.0,1008.0,12.6,60.9,24.7,0.4,0.0,311.8,1006.0,True


In [3]:
outdir = "../data/processed"
df = pd.read_csv(outdir + "/dataset1.csv")

In [4]:
df.head()

,code,Fecha,T_14,Hum_14,DVin_14,VVin_14,Prec_14,RadSol_14,PATM_14h,T_19,Hum_19,DVin_19,VVin_19,Prec_19,RadSol_19,PATM_19h,Target
0,13,2010-06-11,7.2,66.7,61.4,0.0,0.0,146.0,1006.0,3.8,84.7,146.5,0.0,0.0,0.0,1005.0,True
1,13,2010-06-12,14.0,48.6,315.7,0.0,0.0,246.3,1008.0,9.7,66.0,266.1,0.0,0.0,0.0,1013.0,False
2,13,2010-06-13,8.6,82.4,228.0,0.0,0.0,118.5,1011.0,6.3,95.5,141.3,0.0,0.0,0.0,1011.0,False
3,13,2010-06-14,6.0,84.9,53.2,0.0,0.2,53.6,1014.0,3.7,94.2,114.6,0.0,0.4,0.0,1019.0,False
4,13,2010-06-15,0.8,89.7,240.3,0.0,0.0,39.6,1026.0,0.7,89.3,144.7,0.0,0.0,0.0,1027.0,True


In [5]:
import seaborn as sns

In [ ]:
sns.pairplot(df, columns=df.columns[2:-1])